In [ ]:
# !pip install openai
# !pip install regex
# !pip install appdirs
# !pip install jinja2

In [ ]:
from piidetection import OpenAI
from piidetection import Piidetector

In [ ]:
import json

In [ ]:
model = OpenAI(api_key="xxxx")
nlp_prompter = Piidetector(model)

In [ ]:
data = json.load(open("E://PII_Detection_Project//PII_Detection_Project-b05ffa9d799699be9ad7452949d59b1db1335667//examples//data//binary_pii.json",'r'))

In [ ]:
len(data)

19

In [ ]:
examples = []
for sample in data[:3]:
    print(sample,"\n")
    examples.append((sample['text'],sample['labels']))

{'text': "Tom Smith, a 28-year-old man from California, had his wallet stolen while shopping at the mall. His driver's license, credit cards, and social security card were inside. He immediately reported the theft to the police and his bank.", 'labels': 'pii', 'score': '', 'complexity': ''} 

{'text': 'The sun was shining bright as Mary walked through the park. She loved the feeling of the warm sun on her skin and the smell of fresh cut grass. She decided to lay down on the grass and take a nap.', 'labels': 'non-pii', 'score': '', 'complexity': ''} 

{'text': 'Samantha Jones, a 35-year-old woman from New York, received a phone call from someone claiming to be from her bank. They asked for her name, address, and social security number to verify her account. She became suspicious and hung up, then reported the call to her bank.', 'labels': 'pii', 'score': '', 'complexity': ''} 



In [ ]:
prompt = nlp_prompter.generate_prompt('binary_classification.jinja',
                                      label_0="pii",
                                      label_1="non-pii",
                                      examples=examples,
                                      text_input="Michael Davis, a 20-year-old man from Texas, applied for a student loan. He provided his name, address, social security number, and other personal information to the lender. Unfortunately, the lender was hacked, and his information was exposed in a data breach. He had to monitor his credit report and freeze his accounts to prevent further damage.",
                                     description="Binary Classificaion System")
print(prompt)

Binary Classificaion System
You are a highly intelligent and accurate Binary Classification system. You take Passage as input and classify that as either pii or non-pii Category. Your output format is only [{'C':Category}] form, no other form.

Examples:

Input: Tom Smith, a 28-year-old man from California, had his wallet stolen while shopping at the mall. His driver's license, credit cards, and social security card were inside. He immediately reported the theft to the police and his bank.
Output: [{'C': 'pii' }]

Input: The sun was shining bright as Mary walked through the park. She loved the feeling of the warm sun on her skin and the smell of fresh cut grass. She decided to lay down on the grass and take a nap.
Output: [{'C': 'non-pii' }]

Input: Samantha Jones, a 35-year-old woman from New York, received a phone call from someone claiming to be from her bank. They asked for her name, address, and social security number to verify her account. She became suspicious and hung up, then 

In [ ]:
import wandb

# initialize W&B project
wandb.init(project='binary_classification')

# set up W&B configuration
wandb.config.update({
    'learning_rate': 0.01,
    'optimizer': 'Adam',
    'batch_size': 32
})

# wrap nlp_prompter.fit() with W&B logging
with wandb.run():
    output = nlp_prompter.fit('binary_classification.jinja',
                             label_0="pii",
                             label_1="non-pii",
                             examples=examples,
                             text_input="Michael Davis, a 20-year-old man from Texas, applied for a student loan. He provided his name, address, social security number, and other personal information to the lender. Unfortunately, the lender was hacked, and his information was exposed in a data breach. He had to monitor his credit report and freeze his accounts to prevent further damage.",
                             model_name="text-davinci-003",
                             callbacks=[wandb.keras.WandbCallback()])

    # log training and validation loss and accuracy
    wandb.log({'train_loss': output['train_loss'], 'val_loss': output['val_loss'],
               'train_acc': output['train_acc'], 'val_acc': output['val_acc']})

    # save the trained model weights and architecture
    wandb.save('model.h5')

In [ ]:
# output = nlp_prompter.fit('binary_classification.jinja',
#                  label_0="pii",
#                  label_1="non-pii",
#                  examples=examples,
#                  text_input="Michael Davis, a 20-year-old man from Texas, applied for a student loan. He provided his name, address, social security number, and other personal information to the lender. Unfortunately, the lender was hacked, and his information was exposed in a data breach. He had to monitor his credit report and freeze his accounts to prevent further damage.",
#                  model_name="text-davinci-003")

In [ ]:
eval(output['text'].strip())

[{'C': 'pii'}]